In [1]:
from dotenv import load_dotenv
load_dotenv()

import time, os, configparser
import pandas as pd
import datetime as dt
from shared_resources import ib, add_log, start_event
from supabase import create_client

url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_SERVICE_ROLE_KEY")
supabase = create_client(url, key)

PARAMS = {
    1:{'name':'Monthly Trendfilter','value': 10,
       'description':"The 10M SMA Trendfilter is used as a sell signal if the price drops below."},
    2:{'name':"Structural Trendfilter",'value':50,
       'description':"""This filter is used to re-enter the market if the price is below the monthly trendfilter
                        and was below the structural trend, but price just crossed this structural trendline from below."""},
    3:{'name':'Equity Weight','value':30,'description':'Weight for equity allocation'},
    4:{'name':'Fixed Income Weight','value':90,'description':'Weight for FI allocation'},
}


In [13]:
supabase.table("strategies").select().execute()
# supabase.table("strategies").insert({""})


2023-11-11 22:35:43,016:INFO - HTTP Request: HEAD https://lbvljropqikzznsdsicc.supabase.co/rest/v1/strategies "HTTP/1.1 200 OK"


APIResponse[~_ReturnT](data=[], count=0)

2023-11-11 23:06:04,753:INFO - Warning 2103, reqId -1: Market data farm connection is broken:cashfarm
2023-11-11 23:06:05,363:INFO - Warning 2104, reqId -1: Market data farm connection is OK:cashfarm
2023-11-11 23:45:00,668:ERROR - Peer closed connection.


In [ ]:
def get_strategy_details_from_ini(strategy_id):
        config = configparser.ConfigParser()
        config.read('settings.ini')

        strategy_section = f"Strategy{strategy_id}"
        if strategy_section in config:
            strategy_name = config[strategy_section].get('name', '')
            strategy_symbol = config[strategy_section].get('symbol', '')
            strategy_description = config[strategy_section].get('description', '')
            strategy_allocation = config[strategy_section].get('allocation', '')

            # Create a dictionary with strategy details
            strategy_details = {
                "id": strategy_id,
                "name": strategy_name,
                "description": strategy_description,
                "target_weight": float(strategy_allocation),
                # Add more fields as necessary
            }

            return strategy_details
        else:
            # Handle the case where the strategy does not exist in settings.ini
            print(f"Strategy with ID {strategy_id} not found in settings.ini.")
            return None

In [ ]:

class Strategy:
    def __init__(self,ib):
        self.running = True  # This flag controls the while loop in the run method
        self.ib = ib
        self.strategy_id = 1
        self.universe = {1:{'symbol':"IUSQ",'weight':0.3},
                         2:{'symbol':"SYB3",'weight':0.35},
                         3:{'symbol':"IBTE",'weight':0.35}}                   
        try:
            # Load Cash Value from Supabase databank 
            self.cash = 1/0
        except:
            # Get Cash Value from settings.ini weight * total portfolio cash
            weight = 0.3
            total_cash = float([line for line in ib.accountSummary() if line.tag == 'TotalCashValue'][0].value)
            self.cash = total_cash * weight

    def initialize_strategy_in_supabase(self):
        # Load strategy details from settings.ini
        strategy_details = self.get_strategy_details_from_ini(self.strategy_id)

        # Check if the strategy exists in Supabase
        strategies_table = supabase.table("strategies")
        strategy_data = strategies_table.select("*").eq("id", self.strategy_id).execute()

        if strategy_data.error:
            # Handle error (e.g., log or raise an exception)
            add_log(f"Error fetching strategy: {strategy_data.error}")

        if not strategy_data.data:
            # If strategy does not exist, insert it into Supabase
            response = strategies_table.insert(strategy_details).execute()
            if response.error:
                # Handle error (e.g., log or raise an exception)
                add_log(f"Error inserting strategy: {response.error}")
            else:
                add_log(f"Strategy {strategy_details['name']} added to Supabase.")
        else:
            add_log(f"Strategy {strategy_details['name']} exists in Supabase.")


    def warm_up(self, contract):
        '''Downloading historical data for each asset in the universe'''

        def last_day_of_month(any_day):
            # The day 28 exists in every month. 4 days later, it's always next month
            next_month = any_day.replace(day=28) + dt.timedelta(days=4)
            # subtracting the number of the current day brings us back one month
            return (next_month - dt.timedelta(days=next_month.day)).day

        df = util.df(ib.reqHistoricalData(contract,endDateTime='',durationStr='5 Y',barSizeSetting='1 day',whatToShow='TRADES',useRTH=True,formatDate=1))
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)

        # Add a column that identifies the last trading day of the month
        df['last_day_of_month'] = df.index.map(last_day_of_month)
    


    def buy_engine(self):
        # Placeholder buy/sell engines
        add_log("Selecting buy/sell engines...")

        for asset in self.universe:
            symbol = self.universe[asset]['symbol']
            contract = Stock(symbol, 'SMART', 'EUR')

            # is asset invested?
            if symbol in [position.contract.symbol for position in ib.positions()]:
                invested = True

                # is the asset weight below its allocation range (0.2 - 0.3?)

            dfd, dfm = self.warm_up(symbol)

    def select_portfolio(self):
        # Placeholder for portfolio selection
        add_log("Selecting portfolio...")

    def size_initial_position(self):
        # Placeholder for sizing initial position
        add_log("Sizing initial position...")

    def execute_initial_positions(self):
        # Placeholder for executing initial positions
        add_log("Executing initial positions...")

    def check_ongoing_position_size(self):
        # Check the size of ongoing positions
        add_log("Checking ongoing position size...")
        positions = ib.positions()
        for position in positions:
            # Log the position details; implement any size checking logic as required
            add_log(f"Position in {position.contract.symbol}: {position.position}")

    def execute_position_adjustments(self):
        # Placeholder for position adjustments
        add_log("Executing position adjustments...")

    def run(self):
        start_event.wait()  # Wait for the start event to be set
        
        self.select_buy_sell_engines()
        self.select_portfolio()
        self.size_initial_position()
        self.execute_initial_positions()
        while self.running:
            self.check_ongoing_position_size()
            self.execute_position_adjustments()
            time.sleep(10)  # Wait for 10 seconds before checking again

def strategy1(Strategy):
    strategy = Strategy()
    try:
        add_log("Strategy1 Thread Started")
        strategy.run()
    except KeyboardInterrupt:
        add_log("Strategy1 Thread Stopped")
        strategy.running = False  # Set running to False to stop the loop


